# Hack-Heist 2022 - Implimentation Dominators

### Auto Generating the Summary and action items from the meeting

#### Team Members Loui, Madhukar and Farah

------------------------

Execution Instructions: Please follow the steps 1,2 and 3 mentioned in the below cells

### Video file to Audio Converter Implimentation

In [ ]:
!pip3 install ffmpeg moviepy
!pip3 install IPython

In [ ]:
import moviepy.editor as mp
import IPython #to play the audio file in notebook

def convert_video_to_audio(videofile_name,audiofile_name):
    print("converting Video to Audio file..")
    my_clip=mp.VideoFileClip(videofile_name)
    my_clip.audio.write_audiofile(audiofile_name)

### Audio to Text Converter Implimentation

In [ ]:
!pip3 install pydub
!pip3 install SpeechRecognition
!pip3 install audioread

In [ ]:
import sys
import speech_recognition as sr
from pydub import AudioSegment
from pydub.silence import split_on_silence
import os



#This can convert largefiles. It will split the main file into small chunks and convert to text
def large_audio_to_text(audio_file):
    converted_text = ""
    error=""
    r = sr.Recognizer()
    print("converting Audio file {} to Text file..".format(audio_file))
    sound = AudioSegment.from_wav(audio_file)
    audio_chunks = split_on_silence(sound, min_silence_len = 1000, silence_thresh = sound.dBFS-16, keep_silence=500)
    folder_name = "split_audio"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    for i, chunk in enumerate(audio_chunks, start=1):
        split_audio_file = os.path.join(folder_name,f"chunk{i}.wav")
        #print(split_audio)
        chunk.export(split_audio_file,format="wav")
        #audio_to_text(split_audio)
        with sr.AudioFile(split_audio_file) as source:
            #r.adjust_for_ambient_noise(source)
            audio = r.record(source)
            try:
                text = r.recognize_google(audio)
            except sr.UnknownValueError as e:
                #print("{} : error:: {}".format( split_audio_file, str(e)))
                error+=str(e)
            else:
                text = f"{text.capitalize()}. "
                #print("{} : {}".format(split_audio_file, text))
                converted_text += text
    print("\n \n")
    print(converted_text)
    if ".wav" in audio_file:
        textfile = audio_file.strip(".wav")+""+"_ConvertedText.txt"
    else:
        textfile = "_ConvertedText.txt"
    with open(textfile, "w+") as text_file:
        text_file.write(converted_text)
    print("\n Completed and placed text in {} \n".format(textfile))
    return textfile



###  Processing text with NLP and spaCy

In [ ]:
!pip3 install spacy
!python3 -m spacy download en_core_web_sm

In [ ]:
import spacy

def process_text(textfile_name):
    nlp = spacy.load('en_core_web_sm')
    textfile = open(textfile_name).read()
    doc = nlp(textfile)
    #print(doc)
    sentences = list(doc.sents)
    action =[]
    summary = []
    collaborators =[]
    
    for sentence in sentences:
        for word in sentence:
            if not (word.is_stop or word.is_punct):
                if word.pos_ == 'NOUN'and (word.text in ['request']):
                    summary.append(sentence)
                if word.pos_ == 'VERB' and (word.text in ['send','update']):
                    action.append(sentence)
                if word.pos_ == 'PROPN':
                    collaborators.append(word.text)
                    
    print("\n Collaborators: \n")
    print(collaborators)
    
    print("\n Summary of the meeting: \n")
    print('\n'.join([line.text for line in summary]))
               
    print("\n Action items of the meeting: \n")
    print('\n'.join([line.text for line in action]))

### Step1: Converting Video to Audio

In [ ]:

videofile_name = "Meeting_Recording.mp4"
audiofile_name = "Audio_File.wav"

convert_video_to_audio(videofile_name,audiofile_name)
print("playing the converted audio file..")
IPython.display.Audio(audiofile_name)



### Step2: Converting Audio to text

In [ ]:

textfile_name = large_audio_to_text(audiofile_name)


### Step3: Processing Text

In [ ]:
process_text(textfile_name)